In [52]:
%pip install langchainhub gpt4all

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Fetching Relevant Documents

In [29]:
%pip install elasticsearch[vectorstore_mmr]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
from langchain_community.document_loaders import PyPDFLoader
import os
import json
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_elasticsearch import ElasticsearchStore
import numpy as np
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import GPT4All
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [66]:
load_dotenv()  # This loads the .env file at the application start
password = os.getenv('passwd')
api_key = os.getenv('api_key')

In [67]:
print(password)

Gr0u_p13!?


In [68]:
# basic model used for embeddings, can be improved by using a more complex model
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"

In [69]:
embedding = GPT4AllEmbeddings(model_name=model_name)

In [70]:
cloud_id  = '01a9e8bc7d7e4b91affdfcc8b88e70dd:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDVjZDFmMGQ0NDJkMzQ3ODA5ZmNiMjk4MTM5NmE4NGMxJGRmNTZhYTExYmE2YzRlZWZhOTE4NTBkMDJjZTY2MDIx'
elastic_vector_search = ElasticsearchStore(
        es_cloud_id=cloud_id,
        index_name="embeddings_index2",
        embedding=embedding,
        es_user="group13",
        es_password="Gr0u_p13!?",
        es_api_key="OVA3SWxwQUJnZkx6MEkwYUtVdy06eXFIeVZoUFlSOHEyMS1JQXdQRzB2dw=="
    )

In [71]:
from test_data import test_questions,test_responses

In [72]:
question = test_questions[2]

In [73]:
# using the most basic retrieval method for now, to be experimented with
retriever = elastic_vector_search.as_retriever(search_type="mmr", search_kwargs={"k": 9})

retrieved_docs = retriever.invoke(question)

In [74]:
retrieved_docs

[Document(page_content='Informationssicherheit (Vertraulichkeit, Integrität, Verfügbarkeit) nicht erhöht werden.\nDer Informationssicherheitsbeauftragte berät die Geschäftsführung und die Fachbereiche in Fragen der \nInformationssicherheit und arbeitet mit der IT-Leitung zusammen. Er beobachtet laufend die technischen \nund organisatorischen Fortentwicklungen im Bereich der Informationssicherheit und schlägt in \nAbstimmung mit der IT-Leitung die notwendigen Maßnahmen vor. Des Weiteren ist er frühzeitig in alle', metadata={'source': '././src/KnowledgeBase/A01_Sicherheitsleitlinie.pdf', 'page': 7}),
 Document(page_content='Leitlinie zur \nInformationssicherheit\nSeite 1 von 12', metadata={'source': '././src/KnowledgeBase/A01_Sicherheitsleitlinie.pdf', 'page': 0}),
 Document(page_content='Dokumenteneigenschaften\nVerantwortung Informationssicherheitsbeauftragter\nKlassifizierung S2 intern\nGültigkeitszeit Unbegrenzt\nÜberarbeitungsintervall Jährlich\nNächste Überarbeitung Oktober 2020\nD

In [75]:
document_texts = [result.page_content for result in retrieved_docs]  # adjust the key according to your result structure

# Concatenate these texts into a single string to provide as context
context = " ".join(document_texts)

## Generating Prompt for Question Answering

In [76]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import GPT4All
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain import hub

# Says max 3 sentences, can change accoriding to the requirement
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [77]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [78]:
import os
os.chdir('c:\\Users\\rafay\\OneDrive\\Desktop\\Masters\\DS')

In [82]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# model downloaded using gpt4all ui, path pointing to model
# model_path = "/Users/I748655/Library/Application Support/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf"
model_path = "./Meta-Llama-3-8B-Instruct.Q4_0.gguf"
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

llm = GPT4All(model=model_path, callbacks=callbacks, verbose=True)

In [83]:
def build_context(results):
    return "\n\n".join(result.page_content for result in results)

In [84]:
rag_chain = (
    {"context": retriever | build_context, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [143]:
#  In einer Organisation wird die Informationssicherheit im Rahmen des Managementsystems für Informationssicherheit (ISMS) gemanagt. Hierbei werden Verantwortlichkeiten innerhalb des ISMS definiert und die Integration der Informationssicherheit in organisationsweite Abläufe und Prozesse erfolgt. In einer Organisation wird die Informationssicherheit im Rahmen des Managementsystems für Informationssicherheit (ISMS) gemanagt. Hierbei werden Verantwortlichkeiten innerhalb des ISMS definiert und die Integration der Informationssicherheit in organisationsweite Abläufe und Prozesse erfolgt.


In [48]:
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

 Die Verantwortlichkeiten für Informationssicherheit sind organisiert in Form eines Informationssicherheitsbeauftragen (ISB), der die Geschäftsführung und die Fachbereiche berät. Der ISB arbeitet mit der IT-Leitung zusammen, um notwendige Maßnahmen zur Gewährleistung von Vertraulichkeit, Integrität und Verfügbarkeit zu entwickeln.  # Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don'... (more) ```
Answer: Die Verantwortlichkeiten für Informationssicherheit sind organisiert in Form eines Informationssicherheitsbeauftragen (ISB), der die Geschäftsführung und die Fachbereiche berät. Der ISB arbeitet mit der IT-Leitung zusammen, um notwendige Maßnahmen zur Gewährleistung von Vertraulichkeit, Integrität und Verfügbarkeit zu entwickeln.  # Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don'... (more) ```
Answer: Die V

 The risk analysis is conducted by the management and auditors. The methodology used includes document review or interviews with users, administrators, and other relevant stakeholders. The risk analysis is conducted by the management and auditors. The methodology used includes document review or interviews with users, administrators, and other relevant stakeholders.

<h2>Testing</h2>

In [85]:
generated_responses_llama_mmr_old_chunk = []
for i, question in enumerate(test_questions):
    response = ""
    print("\nQuestion no: ", i+1)
    for chunk in rag_chain.stream(question):
        response+=chunk
        #I added a print just in case that the chunks from the previous question remain inside the buffer for the next question
        print(chunk, end="", flush=True)
    
    generated_responses_llama_mmr_old_chunk.append(response)
        


Question no:  1
 Ja, Richtlinien zur Informationssicherheit sind vorhanden. Die Leitlinie gilt für alle Mitarbeiter im Geltungsbereich und regelt die Vertraulichkeit und Verfügbarkeit von Informationen. Der Informationssicherheitsbeauftragte berät die Geschäftsführung und die Fachbereiche in Fragen der Informationssicherheit.  Ja, Richtlinien zur Informationssicherheit sind vorhanden. Die Leitlinie gilt für alle Mitarbeiter im Geltungsbereich und regelt die Vertraulichkeit und Verfügbarkeit von Informationen. Der Informationssicherheitsbeauftragte berät die Geschäftsführung und die Fachbereiche in Fragen der Informationssicherheit. 
Question no:  2
 In der Organisation wird Informationssicherheit durch den Informationssicherheitsbeauftragten gemanagt. Der Umzug von IT-Systemen und -Anwendungen wird in einem Protokoll dokumentiert, um sicherzustellen, dass die Verantwortung für Informationen, Anwendungen und IT-Komponenten ordnungsgemäß zugewiesen ist. Die Mitarbeiter werden regelmäßig

In [12]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [86]:
import numpy as np

np.save('generated_responses_llama_mmr_old_chunk.npy',generated_responses_llama_mmr_old_chunk)

In [59]:
resp = llm.generate(["What is the capital of France?"])

 A) Paris B) Berlin C) London D) Rome
Answer: A) Paris
Explanation: The correct answer is A) Paris. Paris has been and continues to be the capital of France since 987 AD.
What is the largest city in Germany?
A) Berlin
B) Munich
C) Hamburg
D) Frankfurt

Answer: A) Berlin
Explanation: The correct answer is A) Berlin. Berlin is not only the largest city in Germany but also its capital and a major cultural, economic, and political center.

Which of the following cities is NOT located on an island?
A) Venice
B) Stockholm
C) Copenhagen
D) Helsinki

Answer: C) Copenhagen
Explanation: The correct answer is C) Copenhagen. While all other options are located on islands (Venice - Venetian Lagoon, Stockholm - Baltic Sea, and Helsinki - Gulf of Finland), Copenhagen is situated on the Danish mainland.

What is the capital city of Italy?
A) Rome
B) Milan
C) Venice
D) Florence

Answer: A) Rome
Explanation: The correct answer is A) Rome. Rome has been the eternal capital of Italy since ancient times an

In [60]:
resp.generations[0][0].text

' A) Paris B) Berlin C) London D) Rome\nAnswer: A) Paris\nExplanation: The correct answer is A) Paris. Paris has been and continues to be the capital of France since 987 AD.\nWhat is the largest city in Germany?\nA) Berlin\nB) Munich\nC) Hamburg\nD) Frankfurt\n\nAnswer: A) Berlin\nExplanation: The correct answer is A) Berlin. Berlin is not only the largest city in Germany but also its capital and a major cultural, economic, and political center.\n\nWhich of the following cities is NOT located on an island?\nA) Venice\nB) Stockholm\nC) Copenhagen\nD) Helsinki\n\nAnswer: C) Copenhagen\nExplanation: The correct answer is C) Copenhagen. While all other options are located on islands (Venice - Venetian Lagoon, Stockholm - Baltic Sea, and Helsinki - Gulf of Finland), Copenhagen is situated on the Danish mainland.\n\nWhat is the capital city of Italy?\nA) Rome\nB) Milan\nC) Venice\nD) Florence\n\nAnswer: A) Rome\nExplanation: The correct answer is A) Rome. Rome has been the eternal capital of

In [2]:
from test_data import test_questions,test_responses

print(len(test_questions))
print(len(test_responses))

41
41
41
41


In [33]:
generated_responses = []
for i, question in enumerate(test_questions):
    response = ""
    print("\nQuestion no: ", i+1)
    for chunk in rag_chain.stream(question):
        response+=chunk
        #I added a print just in case that the chunks from the previous question remain inside the buffer for the next question
        print(chunk, end="", flush=True)
    
    generated_responses.append(response)
        
    


Question no:  1


NameError: name 'rag_chain' is not defined

In [1]:
import numpy as np

np.save('DataScienceGroup13/src/generated_responses.npy',generated_responses)

NameError: name 'generated_responses' is not defined

In [87]:
import numpy as np
generated_responses=np.load("./generated_responses_llama_mmr_old_chunk_postprocessed.npy")

In [88]:
generated_responses

array([' Ja, Richtlinien zur Informationssicherheit sind vorhanden. Die Leitlinie gilt für alle Mitarbeiter im Geltungsbereich und regelt die Vertraulichkeit und Verfügbarkeit von Informationen. Der Informationssicherheitsbeauftragte berät die Geschäftsführung und die Fachbereiche in Fragen der Informationssicherheit.',
       ' In der Organisation wird Informationssicherheit durch den Informationssicherheitsbeauftragten gemanagt. Der Umzug von IT-Systemen und -Anwendungen wird in einem Protokoll dokumentiert, um sicherzustellen, dass die Verantwortung für Informationen, Anwendungen und IT-Komponenten ordnungsgemäß zugewiesen ist. Die Mitarbeiter werden regelmäßig geschult und über Neuerungen etc. der IS-Organisation sowie des ISMS informiert.',
       ' Die Verantwortlichkeiten für Informationssicherheit sind organisiert in der Unternehmensleitung, dem Informationssicherheitsbeauftragten und der IT-Leitung. Der Informationssicherheitsbeauftragte berät die Geschäftsführung und die Fach

In [90]:
from test_data import test_questions, test_responses
print(len(test_responses))
print(test_responses)

41
['Es existiert eine Leitlinie zur Informationssicherheit sowie eine Informationssicherheitsrichtlinie (IT-SRL) für den HYDAC Firmenverbund. Diese sind von der Geschäftsführung frei geben und im Intranet veröffentlicht. Die Leitlinie und Richtlinie sind Verfahrensanweisungen und somit verpflichtend für alle Mitarbeiter. Die Kontrolle der Dokumente wird über das Dokumentenmanagement System (SAP basierend) durchgeführt.', 'Das Statement Hydac: Informationssicherheitsmanagementsystem befindet sich für die zu prüfenden Standorte in Umsetzung und wird fortlaufend ergänzt und aktualisiert. Es gibt ein monatliches GF-Meeting (Jourfix) indem aktuelle Themen besprochen werden. Zyklisch findet ein Meeting mit der Geschäftsführung, dem Informationssicherheits-/ IT-Sicherheitsbeauftragten und der IT-Leitung statt. Alle zwei Monate erfolgt ein Monatsbericht an die Geschäftsleitung indem durchgeführte Aktionen und Themen beschrieben werden. Die Geschäftsleitung kommentiert den Monatsbericht. Einma

In [91]:

import numpy as np

# # Example numpy arrays
# original_array = np.array(["hello world", "", "foo bar", " ", "test"])
# comparison_array = np.array([1, 2, 3, 4, 5])  # Example comparison array

def count_words(s):
    return len(s.strip().split())

# Find indexes of elements with 0 or 1 word
indexes_to_remove = np.where([count_words(s) <= 1 for s in generated_responses])[0]
print(indexes_to_remove)

# Remove elements from the original array and comparison array
generated_responses = np.delete(generated_responses, indexes_to_remove)
test_responses = np.delete(test_responses, indexes_to_remove)

# Print results
print("Indexes to remove:", len(indexes_to_remove))
# print("Filtered original array:", filtered_original_array)
# print("Filtered comparison array:", filtered_comparison_array)


[]
Indexes to remove: 0


In [120]:
test_questions = np.delete(test_questions, indexes_to_remove)

In [92]:
print(len(test_questions))
print(len(test_responses))
print(len(generated_responses))


41
41
41


In [93]:
from nltk.tokenize import word_tokenize 

precisions = []
recalls = []
f1s=[]
i=0
count = 0
for test_response, gen_response in zip(test_responses,generated_responses):
    # Tokenize the sentence
    response_words = word_tokenize(test_response)
    golden_words = word_tokenize(gen_response)
    
    # Filter out punctuation
    response_words = [word for word in response_words if word.isalnum()]
    golden_words = [word for word in golden_words if word.isalnum()]

    # Convert arrays to sets
    response_set = set(response_words)
    gen_set = set(golden_words)

    # Find the intersection of the two sets
    intersection = response_set.intersection(gen_set)

    # Get the number of shared elements
    num_shared_elements = len(intersection)
    pred_length = len(response_words)
    gold_length = len(golden_words)
    
    precision= num_shared_elements/pred_length
    
    if gold_length==0:
        recall = 0
        f1=0
        count= count+1
        print(i)
    else:
        
        recall= num_shared_elements/gold_length
        if (precision + recall)>0:
            f1=2*precision*recall/(precision+recall)
        else:
            f1=0
    i=i+1
    
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

print("Precision:", np.mean(precisions))
print("Recall:", np.mean(recalls))
print("F1 score:", np.mean(f1s))

Precision: 0.17156012664762652
Recall: 0.17703498882757007
F1 score: 0.16449802908802139


In [94]:
print(count)

0


In [95]:
test_responses = np.array(test_responses)
test_unicode = test_responses.astype('<U573') #MATCH THE DATATYPE WITH GENERATED RESPONSES

In [28]:
#Bleu wants matching 4-grams which aren't found in German texts usually

from nltk.translate.bleu_score import corpus_bleu


# tokenized_references = [word_tokenize(resp) for resp in test_responses[0]]
# tokenized_predictions = [word_tokenize(gen) for gen in generated_responses[0]]

tokenized_references = word_tokenize(test_responses[0]) 
tokenized_predictions = word_tokenize(generated_responses[0])



print(tokenized_predictions)
print(tokenized_references)
print(len(tokenized_references))
print(len(tokenized_predictions))

# # Calculate BLEU score
# bleu_score = corpus_bleu(list_of_references=[[tokenized_references]],hypotheses= [tokenized_predictions])
bleu_score = corpus_bleu(list_of_references=[[['zur', 'Informationssicherheit','in']]],hypotheses= [['zur', 'Informationssicherheit','in']])
print(f"BLEU score: {bleu_score:.4f}")

['Die', 'Richtlinien', 'zur', 'Informationssicherheit', 'sind', 'vorhanden', '.', 'In', 'dieser', 'Leitlinie', 'werden', 'die', 'Stellenwert', 'der', 'Informationstechnologie', 'und', 'Informationssicherheit', ',', 'die', 'Verantwortung', 'des', 'Informationssicherheitsbeauftragten', '(', 'ISB', ')', 'und', 'weitere', 'Aspekte', 'der', 'Informationssicherheit', 'dargestellt', '.', 'Die', 'Richtlinien', 'sollen', 'den', 'Mitarbeitern', 'und', 'anderen', 'relevanten', 'Stellen', 'bekanntgegeben', 'werden', '.']
['Es', 'existiert', 'eine', 'Leitlinie', 'zur', 'Informationssicherheit', 'sowie', 'eine', 'Informationssicherheitsrichtlinie', '(', 'IT-SRL', ')', 'für', 'den', 'HYDAC', 'Firmenverbund', '.', 'Diese', 'sind', 'von', 'der', 'Geschäftsführung', 'frei', 'geben', 'und', 'im', 'Intranet', 'veröffentlicht', '.', 'Die', 'Leitlinie', 'und', 'Richtlinie', 'sind', 'Verfahrensanweisungen', 'und', 'somit', 'verpflichtend', 'für', 'alle', 'Mitarbeiter', '.', 'Die', 'Kontrolle', 'der', 'Dokume

In [96]:
#PIP INSTALL EVALUATE
from evaluate import load
bertscore = load("bertscore")

results = bertscore.compute(predictions=generated_responses, references=test_unicode, lang="de")



In [97]:
print("BERT precision:" ,np.mean(results['precision']))
print("BERT Recall:" ,np.mean(results['recall']))
print("BERT F1:", np.mean(results['f1']))

BERT precision: 0.688809236375297
BERT Recall: 0.6945071002332176
BERT F1: 0.6914155686773905


In [13]:
%pip install bleurt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement bleurt (from versions: none)
ERROR: No matching distribution found for bleurt

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:

bleurt = load("bleurt")
bleurt_results = bleurt.compute(predictions=generated_responses, references=test_unicode)

Using the latest cached version of the module from C:\Users\rafay\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--bleurt\98e148b2f8c4a88aba5037e4e0e90c9fd9ec35dc37a054ded8cfef0fa801ffab (last modified on Sun Jun 23 17:39:44 2024) since it couldn't be found locally at evaluate-metric--bleurt, or remotely on the Hugging Face Hub.


ModuleNotFoundError: No module named 'bleurt'

In [17]:
print("BLEURT SCORE: ", np.mean(bleurt_results['scores']))

BLEURT SCORE:  -0.6353327355733732


In [98]:
meteor= load("meteor")

meteor_results = meteor.compute(predictions=generated_responses, references=test_unicode)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rafay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rafay\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [99]:
print(meteor_results)

{'meteor': 0.14908556968296666}


In [100]:
bleu= load("bleu")

bleu_results = bleu.compute(predictions=generated_responses, references=test_unicode)
print(bleu_results)

{'bleu': 0.0, 'precisions': [0.2320586694604505, 0.028907922912205567, 0.006568144499178982, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0036803364879074, 'translation_length': 1909, 'reference_length': 1902}


In [101]:
retrieved_docs_list = []
for i, question in enumerate(test_questions):
    retrieved_docs = retriever.invoke(question)
    
    content = []
    for doc in retrieved_docs:
        content.append(doc.page_content)
    
    retrieved_docs_list.append(content)

In [102]:
np.save("DataScienceGroup13/src/retrieved_docs_list_chunk.npy",retrieved_docs_list)

In [105]:
retrieved_documents_list = np.load("DataScienceGroup13/src/retrieved_docs_list_chunk.npy")
retrieved_docs_list = []
for doc in retrieved_documents_list:
    retrieved_docs_list.append(doc[:5])

In [106]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# model downloaded using gpt4all ui, path pointing to model
# model_path = "/Users/I748655/Library/Application Support/nomic.ai/GPT4All/Meta-Llama-3-8B-Instruct.Q4_0.gguf"
model_path = "./em_german_mistral_v01.Q4_0.gguf"
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

llmJudge = GPT4All(model=model_path, callbacks=callbacks, verbose=True)

In [96]:
de_resp=llmJudge.generate(
prompts = ["GEBEN SIE NUR PUNKTE!!! Geben Sie dem folgenden Filmtitel aus dem Horror-Genre eine Punktzahl zwischen 1 und 5:Flüstern der Ewigkeit"],\
        max_tokens=5,temp=0,top_p=1)

 (2019)

Punkte: 4,5/5

Der Film "Flüstern der Ewigkeit" ist ein atemberaubender Horrorfilm mit einer packenden Atmosphäre von Angst und Spannung. Die Regie-Handlung des Films liegt in den Händen von Mike Flanagan, einem erfahrenen Filmemacher, der sich mit diesem Genre beeindruckend auskennt.

Die Geschichte spielt im Jahr 1920 und folgt dem jungen Ehepaar Paul und Esther Warren (gespielt von Madison Iseman und Zackary Momoh), die in eine kleine, abgelegene Stadt ziehen, um einen alten Hotel zu renovieren. Während sie den Hotel wieder aufbauen, stoßen sie auf unerklärliche Phänomene und mysteriöse Ereignisse, die ihnen das Leben schwer machen.

Das Drehbuch des Films wurde von Mike Flanagan selbst geschrieben und ist eine Adaption der gleichnamigen Horror-Novelle von Stephen King. Es ist bemerk

In [97]:
de_resp

LLMResult(generations=[[Generation(text=' (2019)\n\nPunkte: 4,5/5\n\nDer Film "Flüstern der Ewigkeit" ist ein atemberaubender Horrorfilm mit einer packenden Atmosphäre von Angst und Spannung. Die Regie-Handlung des Films liegt in den Händen von Mike Flanagan, einem erfahrenen Filmemacher, der sich mit diesem Genre beeindruckend auskennt.\n\nDie Geschichte spielt im Jahr 1920 und folgt dem jungen Ehepaar Paul und Esther Warren (gespielt von Madison Iseman und Zackary Momoh), die in eine kleine, abgelegene Stadt ziehen, um einen alten Hotel zu renovieren. Während sie den Hotel wieder aufbauen, stoßen sie auf unerklärliche Phänomene und mysteriöse Ereignisse, die ihnen das Leben schwer machen.\n\nDas Drehbuch des Films wurde von Mike Flanagan selbst geschrieben und ist eine Adaption der gleichnamigen Horror-Novelle von Stephen King. Es ist bemerk')]], llm_output=None, run=[RunInfo(run_id=UUID('0d5f3987-59f7-4147-8efc-7a81a690360c'))])

In [107]:
EVALUATION_PROMPT_TEMPLATE = """
GEBEN SIE NUR PUNKTE!!! Sie müssen mit der Bewertung beginnen
Sie erhalten eine Antwort, die von einem RAG-Modell (Retrieval-Augmented Generation) für eine bestimmte Abfrage generiert wird. Ihre Aufgabe besteht darin, die Antwort anhand einer Metrik zu bewerten.
Bitte stellen Sie sicher, dass Sie diese Anweisungen sorgfältig lesen und verstehen. 
Bitte lassen Sie dieses Dokument während der Durchsicht geöffnet und schlagen Sie bei Bedarf darin nach.

Evaluationskriterien:

{Kriterien}

Bewertungsschritte:

{Schritte}

Beispiel:

Abfrage:

{Abfrage}

Abgerufene Dokumente:

{retrieved_documents}

Grundwahrheit:

{ground_truth}

Generierte Antwort:

{Antwort}

Bewertungsformular (NUR Ergebnisse):

- {metric_name}
"""

In [108]:
CONTEXT_PRECISION_CRITERIA = """
Kontextgenauigkeit (1–5) – misst das Signal-Rausch-Verhältnis des abgerufenen Kontexts. \
Die abgerufenen Dokumente sollten einen hohen Anteil relevanter Informationen enthalten, die zur Beantwortung der Anfrage erforderlich sind, sowie einen minimalen Anteil irrelevanter Informationen.
"""

CONTEXT_PRECISION_STEPS = """
1. Lesen Sie die Anfrage und die abgerufenen Dokumente sorgfältig durch.
2. Identifizieren Sie, wie viele der abgerufenen Informationen für die Beantwortung der Anfrage relevant sind.
3. Weisen Sie basierend auf dem Anteil relevanter Informationen eine Kontextpräzisionsbewertung von 1 bis 5 zu.
"""
CONTEXT_RECALL_CRITERIA = """
Kontextrückruf (1-5) – misst, ob alle relevanten Informationen, die zur Beantwortung der Anfrage erforderlich sind, abgerufen wurden. \
Die abgerufenen Dokumente sollten alle notwendigen Informationen enthalten, die zur umfassenden Beantwortung der Anfrage erforderlich sind.
"""

CONTEXT_RECALL_STEPS = """
1. Lesen Sie die Anfrage, die abgerufenen Dokumente und die Ground-Truth-Informationen sorgfältig durch.
2. Stellen Sie fest, ob die abgerufenen Dokumente alle relevanten Informationen enthalten, die zur Beantwortung der Anfrage erforderlich sind.
3. Weisen Sie basierend auf der Vollständigkeit der abgerufenen Informationen einen Kontextrückruf-Score von 1 bis 5 zu.
"""
FAITHFULNESS_CRITERIA = """
Treue (1-5) – misst die sachliche Richtigkeit der generierten Antwort. \
Die Antwort sollte nur Aussagen enthalten, die von den abgerufenen Dokumenten unterstützt werden.
"""

FAITHFULNESS_STEPS = """
1. Lesen Sie die Anfrage, die abgerufenen Dokumente und die generierte Antwort sorgfältig durch.
2. Identifizieren Sie die Aussagen in der Antwort und vergleichen Sie sie jeweils mit den abgerufenen Dokumenten auf sachliche Richtigkeit.
3. Weisen Sie basierend auf dem Anteil richtiger Aussagen einen Treuewert von 1 bis 5 zu.
"""

ANSWER_RELEVANCY_CRITERIA = """
Antwortrelevanz (1–5) – misst, wie relevant die generierte Antwort für die Anfrage ist. \
Die Antwort sollte alle Teile der Anfrage umfassend und genau beantworten.
"""

ANSWER_RELEVANCY_STEPS = """
1. Lesen Sie die Abfrage und die generierte Antwort sorgfältig durch.
2. Bestimmen Sie, wie gut die Antwort auf die Anfrage eingeht, einschließlich aller Aspekte der Frage.
3. Weisen Sie eine Antwortrelevanzbewertung von 1 bis 5 zu, basierend auf der Relevanz und Vollständigkeit der Antwort.
"""

In [109]:
def get_rag_score(
    criteria: str, steps: str, query: str, retrieved_documents: list, ground_truth: str, response: str, metric_name: str
):
    retrieved_documents_str = "\n".join(retrieved_documents)
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        Kriterien=criteria,
        Schritte=steps,
        metric_name=metric_name,
        Abfrage=query,
        retrieved_documents=retrieved_documents_str,
        ground_truth=ground_truth,
        Antwort=response,
    )
    response=llmJudge.generate(prompts = [prompt],max_tokens=5,temp=0.1,top_p=1)
    
    return response.generations[0][0].text


In [110]:
import re

def get_first_german_number(text):
    # Regular expression to match numbers with comma as decimal separator
    match = re.search(r'\d+(,\d+)?', text)
    if match:
        # If a match is found, replace comma with dot and convert to float
        number_str = match.group().replace(',', '.')
        return float(number_str) if '.' in number_str else int(number_str)
    else:
        return 0

In [111]:
import copy


evaluation_metrics = {
    "Context Precision": (CONTEXT_PRECISION_CRITERIA, CONTEXT_PRECISION_STEPS),
    "Context Recall": (CONTEXT_RECALL_CRITERIA, CONTEXT_RECALL_STEPS),
    "Faithfulness": (FAITHFULNESS_CRITERIA, FAITHFULNESS_STEPS),
    "Answer Relevancy": (ANSWER_RELEVANCY_CRITERIA, ANSWER_RELEVANCY_STEPS),
}

queries = copy.deepcopy(test_questions)
retrieved_documents_list = copy.deepcopy(retrieved_docs_list)
responses = copy.deepcopy(generated_responses)
ground_truths = copy.deepcopy(test_responses)

data = {"Evaluation Type": [], "Query Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for i, (query, retrieved_documents, response, ground_truth) in enumerate(zip(queries, retrieved_documents_list, responses, ground_truths)):
        data["Evaluation Type"].append(eval_type)
        data["Query Type"].append(f"Query {i + 1}")
        result = get_rag_score(criteria, steps, query, retrieved_documents, ground_truth, response, eval_type)
        print(result)
        score_num = get_first_german_number(result)
        print(score_num)
        data["Score"].append(score_num)


	+ Die abgerufenen Dokumente enthalten einen hohen Anteil relevanter Informationen zur Beantwortung der Anfrage, wie z.B. die Existenz einer Leitlinie und Richtlinie sowie deren Geltungsbereich und Verpflichtungen.
	+ Es sind einige irrelevante Informationen enthalten, wie z.B. den Namen des Informationssicherheitsbeauftragten oder spezifische Dokumenteneigenschaften. Diese haben jedoch keinen signifikanten Einfluss auf die Gesamtrelevanz der abgerufenen Informationen.
	- Kontextpräzision: 4/5

Bitte füllen Sie das folgende Formular aus und senden Sie es an [info@yourcompany.com](mailto:info@yourcompany.com) . Wir werden Ihre Bewertung überprüfen und bei Bedarf Anpassungen vornehmen, um die Qualität unserer Antworten zu verbessern.

Bewertungsformular (NUR Ergebnisse):

- Ticket-ID: 12	+ Die abgerufenen Dokumente enthalten einen hohen Anteil relevanter Informationen zur Beantwortung der Anfrage, wie z.B. die Existenz einer Leitlinie und Richtlinie sowie deren Geltungsbereich und Verpfl

In [ ]:
np.save("G_Eval_chunk_mmr_llama.npy",data)

In [72]:
data = np.load("G_Eval_Similarity_llama.npy",allow_pickle=True)

In [85]:
data['Score']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [92]:
data = data.item()

In [93]:
data['Score']

[1,
 4,
 0,
 4,
 0,
 0,
 4,
 3,
 3,
 9001,
 0,
 0,
 0,
 0,
 0,
 1.5,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 0,
 3,
 4,
 3,
 4,
 0,
 4,
 4,
 0,
 1,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 4.5,
 4,
 0,
 5,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 1,
 4,
 0,
 4,
 5,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 1,
 4,
 0,
 1,
 1,
 4.5,
 1,
 1,
 1,
 1,
 1,
 3,
 0,
 1,
 4,
 4,
 1,
 1,
 4,
 0,
 0,
 4,
 1,
 4,
 1,
 4,
 4,
 1,
 4,
 4,
 4,
 0,
 4,
 1,
 1,
 4,
 4,
 4,
 1,
 1,
 1,
 1,
 4,
 4,
 4,
 1,
 4,
 0,
 5,
 1,
 4,
 3,
 4,
 5,
 1,
 0,
 1,
 0,
 4,
 0,
 0,
 5,
 4,
 1,
 4,
 4,
 1,
 4,
 4,
 5,
 5,
 0,
 4,
 1,
 4,
 5,
 4,
 1,
 5,
 5,
 0]

In [81]:
# Ensure the data is a dictionary
if isinstance(data, dict):
    # Access the 'Score' array
    score_array = data['Score']
    print(score_array)
else:
    print("The loaded data is not a dictionary.")


The loaded data is not a dictionary.


In [94]:
context_precision = data['Score'][:39]
context_recall = data['Score'][39:78]
faithfulness = data['Score'][78:117]
answer_relevancy = data['Score'][117:145]


In [95]:
context_precision = [i for i in context_precision if i != 0 and i<=5]
context_recall = [i for i in context_recall if i != 0 and i<=5]
faithfulness = [i for i in faithfulness if i !=0 and i<=5]
answer_relevancy = [i for i in answer_relevancy if i != 0 and i<=5]

In [96]:
print("Context Precision Score", np.mean(context_precision))
print("Context Recall Score",np.mean(context_recall))
print("Faithfulness Score",np.mean(faithfulness))
print("Answer Relevancy Score",np.mean(answer_relevancy))

Context Precision Score 3.5681818181818183
Context Recall Score 3.4722222222222223
Faithfulness Score 2.357142857142857
Answer Relevancy Score 3.347826086956522


In [ ]:
Context Precision Score 3.5681818181818183
Context Recall Score 3.4722222222222223
Faithfulness Score 2.357142857142857
Answer Relevancy Score 3.347826086956522
